In [1]:
# mlflow ui --port 6010 --backend-store-uri file:/share/lazy/will/ConstrastiveLoss/Logs
# watch -n 0.5 nvidia-smi

In [23]:
# stuff for making data loading pipelines once we know what the data format is 
import pandas as pd
import seaborn as sns
import uproot

In [1]:
# examples for .h5 and .root
path1 = '/share/lazy/CaloML/CaloGan/caloGAN_case0_v4_50K.root'
path2 = '/share/lazy/CaloML/CaloGan/CaloGan_photons.h5'

dataset = uproot.open(path1)
test = dataset['ecalNT;1']['EnergyDeposit'].array()
sns.heatmap(test[0].reshape(30,30))

spacal_df = pd.read_hdf(path2)

In [58]:
import torch
from torch import nn, optim
from torch.utils.data import DataLoader

from torchvision import datasets, transforms, utils

from VQVAE import VQVAE
from train import train
from utilities import start_mlflow_experiment, Params, save_to_mlflow, count_parameters, load_full_state, select_gpu

from tqdm import tqdm
import mlflow


In [4]:
device = select_gpu(1)

args = Params(32, 10, 4e-4, 256, device)

start_mlflow_experiment('Vector Quantized Variational Autoencoder', 'Calo-ML')


transform = transforms.Compose([
        transforms.Resize(args.size),
        transforms.CenterCrop(args.size),
        transforms.ToTensor(),
        transforms.Normalize([0.5, 0.5, 0.5], [0.5, 0.5, 0.5]),
    ])

# copied from other project and not useful
dataset = datasets.ImageFolder('/share/lazy/will/ConstrastiveLoss/Imgs/color_images/train/', transform=transform)
loader = DataLoader(dataset, batch_size=args.batch_size, shuffle=True, pin_memory = True)

model = VQVAE(channel=128).to(args.device)

optimizer = optim.Adam(model.parameters(), lr=args.lr)

run_name = 'VQVAE'
with mlflow.start_run(run_name = run_name) as run:

    for epoch in range(args.epoch):
        results = train(epoch, loader, model, optimizer, args.device)
        for Dict in results:
            save_to_mlflow(Dict, args)
            
            # this is not working in the save_to_mlflow function for some reason
            mlflow.log_artifact('run_stats.pyt')


NameError: name 'select_gpu' is not defined